# Your First Binary Simulations with POSYDON 🌠

**Tutorial goal:**

In this tutorial, we will run a small population of 10 binaries locally, and explore how to manipulate the output data from your population.

**New concepts:**

- Population ini file

If you haven't done so yet, export the path POSYDON environment variables.
Set these parameters in your `.bash_profile` or `.zshrc` if you use POSYDON regularly.

In [ ]:
%env PATH_TO_POSYDON=/YOUR/POSYDON/PATH/
%env PATH_TO_POSYDON_DATA=/YOUR/POSYDON_DATA/PATH/

## Creating the Initialisation File

----
To run population synthesis with POSYDON, a `population_params.ini` file is required.
This file described how the stellar population is created and what prescriptions and parameters are implemented in specific steps.

POSYDON comes with a default `population_params_default.ini` file found at `PATH_TO_POSYDON/posydon/popsyn` or have a look [here](population_params.ini).

The file is split in three main parts. You can find more details about their properties by clicking on their links.
1. **[SimulationProperties]():**
   - these describe the properties and parameters of different steps in the evolution of a binary systems.
2. **[BinaryPopulation]():** 
   - parameters of the initial sampling of the binary population, such as initial mass function, period distribution, and metallicity.
   - Also contains parameters on how the population is ran, such as how many binaries are kept in memory.
3. **[SavingOutput]()**
   - Describes the data from the binary and each individual star to the output files.

We will copy the default population run parameter file to the current folder.

In [ ]:
import os
import shutil
from posydon.config import PATH_TO_POSYDON

path_to_params = os.path.join(PATH_TO_POSYDON, "posydon/popsyn/population_params_default.ini")
shutil.copyfile(path_to_params, './population_params.ini')

# Creating and running a binary population

The copied `population_params.ini` contains the parameters to run 10 binaries at a metallicity of $Z=10^{-4}$.

If you open the file and scroll down to the **BinaryPopulation** section, you will see how they're defined:

```
metallicity = [0.0001] # [2., 1., 0.45, 0.2, 0.1, 0.01, 0.001, 0.0001]
# In units of solar metallicity
...
number_of_binaries = 10
# int
```

If you like to run a small population in a notebook, you can use the `PopulationRunner` to do this. If you want to run a specific binary instead, have a look at the [Binary Tutorial]().

The `PopulationRunner` class takes the `.ini` file and sets-up a population run.

This will create [BinaryPopulations]() for each metallicity defined in the `.ini` file.
In this case, we can check and see that a single BinaryPopulation is created and contains 10 binaries.

In [1]:
from posydon.popsyn.synthetic_population import PopulationRunner
poprun = PopulationRunner('./population_params.ini')

In [2]:
print('Number of binary populations:',len(poprun.binary_populations))
print('Metallicity:', poprun.binary_populations[0].metallicity)
print('Number of binaries:', poprun.binary_populations[0].number_of_binaries)  

Number of binary populations: 1
Metallicity: 0.0001
Number of binaries: 10


After setting up the population run, we can evolve it. This should take about 20 seconds, but depends on your machine.

In [3]:
poprun.evolve()

FileExistsError: 1e-04_Zsun_population.h5 already exists!
Files were not merged. You can use PopulationRunner.merge_parallel_runs to merge the files manually.

# Inspecting the population: Population class

When you ran the population, you might have seen that a temporary folder with the name `1e-04_Zsun_batches` was created while the binaries were evolved.
This is a temporary folder in which populations are temporarly saved.
After the binary evolution has finished, the binaries in the folder are moved to a single file named `1e-04_Zsun_popululation.h5`. This is done automatically, when you run a population using the `PopulationRunner` class.

The created file contains 3 main components:

1. **history:** the evolution of an individual binary in a pandas DataFrame
2. **oneline:** a single line to describe the initial and final conditions and some one-of parameters, such as the metallicity.
3. **mass_per_metallicity:** some metadata on the population, such as the total simulated mass, the actual underlying mass of the population, and the number of binaries in the file.

The `Population` provides an interface with these components in the file, such that you're able to share the populations runs and can work with large population that do not fit in memory.


<div class="alert alert-block alert-warning"><b>Older Population Files</b> 

If you're using older population files, you can make them compatible with the `Population` class by calling `Population(pop_file, metallicity,ini_file)`, where the `metallicity` is in solar units. You will only need to do this once; afterwards you can initialise the class like normal.<div>

In [4]:
from posydon.popsyn.synthetic_population import Population

In [5]:
pop = Population('1e-04_Zsun_population.h5')

The `pop.mas_per_met` shows some basic information about the population you've just created.


1. The index is the metallicity of your population in solar units.
2. **simulated mass** is the total ZAMS mass that has been evolved in the population.
3. **underlying mass** is the actual mass of the population if one integrates the IMF and period distribution fully.
4. **number_of_systems** shows the 10 systems in the file.

In [6]:
pop.mass_per_metallicity

,simulated_mass,underlying_mass,number_of_systems
0.0001,321.367361,1807.314212,10


There are some additional metadata properties available, such as:
- `metallicities`: the metallicity in absolute metallicity
- `solar_metallicities`: the metallicities in the file in solar metallicity
- `number_of_systems`: the total number of systems in the Population file
- `indices`: the indices of the binaries in the file
- `columns`: the columns available in the `history` and `oneline` dataframes

In [7]:
# you can also access the total number of systems in the file with
print(pop.number_of_systems)

10


## Population.history

`pop.history` loads in the full history of all the binaries into memory.
You can access individual or a selections of the population using several methods:


1. pop.history[5]
2. pop.history[[0,4]]
3. pop.history['time]
4. pop.history.select()

The select function is the most powerfull way to access the binaries, because it allows you to perform selections based on the columns available in the history dataframe.

For example, below we can select on `state == 'RLO1'`, which gives us all the rows with RLO1 occuring.
At the same time, we can request only specific columns, 


In [8]:
pop.history

,state,event,time,orbital_period,eccentricity,lg_mtransfer_rate,step_names,step_times,S1_state,S1_mass,...,S2_he_core_mass,S2_he_core_radius,S2_co_core_mass,S2_co_core_radius,S2_center_h1,S2_center_he4,S2_surface_h1,S2_surface_he4,S2_surf_avg_omega_div_omega_crit,S2_spin
binary_index,,,,,,,,,,,,,,,,,,,,,
0,detached,ZAMS,0.000000e+00,0.896259,0.000000,NaN,initial_cond,0.000000,H-rich_Core_H_burning,12.732339,...,NaN,NaN,NaN,NaN,0.750999,2.490000e-01,NaN,NaN,NaN,NaN
0,contact,oCE1,1.514959e+07,0.442302,0.000000,-1.639589,step_HMS_HMS,0.101655,H-rich_Core_H_burning,10.037747,...,0.000000,0.000000,0.000000,0.000000,0.634120,3.658788e-01,0.750198,0.249789,0.989435,14.356816
0,merged,oMerging1,1.514959e+07,0.442302,0.000000,-1.639589,step_CE,0.000025,H-rich_Core_H_burning,10.037747,...,0.000000,0.000000,0.000000,0.000000,0.634120,3.658788e-01,0.750198,0.249789,0.989435,14.356816
0,merged,CC1,1.960607e+07,NaN,NaN,NaN,step_merged,2.128161,H-rich_Central_C_depletion,14.963983,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,merged,NaN,1.960607e+07,NaN,NaN,NaN,step_SN,0.000620,NS,1.295261,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9,RLO2,oRLO2,6.252890e+06,26.431750,0.000000,NaN,step_detached,2.211219,BH,18.801878,...,15.692803,NaN,0.000000,0.000000,0.000000,4.082629e-01,0.750999,0.248974,0.325257,2.808919
9,RLO2,CC2,6.469990e+06,27.393797,0.000000,-8.605134,step_CO_HMS_RLO,0.109179,BH,18.990408,...,16.199377,0.246243,14.529115,0.145089,0.000000,1.057180e-12,0.332871,0.667128,0.480049,0.511162
9,detached,NaN,6.469990e+06,28.182768,0.151457,NaN,step_SN,0.000457,BH,18.990408,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.001220


In [9]:
# select only binary_index 5
pop.history[5]

,state,event,time,orbital_period,eccentricity,lg_mtransfer_rate,step_names,step_times,S1_state,S1_mass,...,S2_he_core_mass,S2_he_core_radius,S2_co_core_mass,S2_co_core_radius,S2_center_h1,S2_center_he4,S2_surface_h1,S2_surface_he4,S2_surf_avg_omega_div_omega_crit,S2_spin
binary_index,,,,,,,,,,,,,,,,,,,,,
5,detached,ZAMS,0.000000e+00,190.947756,0.0,NaN,initial_cond,0.000000,H-rich_Core_H_burning,9.200760,...,NaN,NaN,NaN,NaN,0.750999,2.490000e-01,NaN,NaN,NaN,NaN
5,RLO1,CC1,3.070595e+07,592.997847,0.0,-5.068581,step_HMS_HMS,0.011373,H-rich_Central_C_depletion,3.588048,...,1.411542e-14,2.053381e-16,1.365520e-14,1.239204e-16,0.336846,6.631524e-01,0.344334,0.655665,0.909267,15.995758
5,disrupted,NaN,3.070595e+07,NaN,NaN,NaN,step_SN,0.000328,NS,1.211756,...,1.411542e-14,2.053381e-16,1.365520e-14,1.239204e-16,0.336846,6.631524e-01,0.344334,0.655665,0.909267,15.995758
5,disrupted,CC2,4.175273e+07,NaN,NaN,NaN,step_disrupted,0.200534,NS,1.211756,...,2.745715e+00,NaN,1.890312e+00,4.697216e-02,0.000000,5.596109e-14,0.734275,0.265702,0.026005,14.843018
5,disrupted,NaN,4.175273e+07,NaN,NaN,NaN,step_SN,0.000434,NS,1.211756,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000
5,disrupted,END,4.175273e+07,NaN,NaN,NaN,step_end,0.000020,NS,1.211756,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000


In [10]:
# select binary 0 and 4
pop.history[[0,4]]

,state,event,time,orbital_period,eccentricity,lg_mtransfer_rate,step_names,step_times,S1_state,S1_mass,...,S2_he_core_mass,S2_he_core_radius,S2_co_core_mass,S2_co_core_radius,S2_center_h1,S2_center_he4,S2_surface_h1,S2_surface_he4,S2_surf_avg_omega_div_omega_crit,S2_spin
binary_index,,,,,,,,,,,,,,,,,,,,,
0,detached,ZAMS,0.000000e+00,0.896259,0.0,NaN,initial_cond,0.000000,H-rich_Core_H_burning,12.732339,...,NaN,NaN,NaN,NaN,0.750999,2.490000e-01,NaN,NaN,NaN,NaN
0,contact,oCE1,1.514959e+07,0.442302,0.0,-1.639589,step_HMS_HMS,0.101655,H-rich_Core_H_burning,10.037747,...,0.000000,0.0,0.00000,0.000000,0.634120,3.658788e-01,0.750198,0.249789,0.989435,14.356816
0,merged,oMerging1,1.514959e+07,0.442302,0.0,-1.639589,step_CE,0.000025,H-rich_Core_H_burning,10.037747,...,0.000000,0.0,0.00000,0.000000,0.634120,3.658788e-01,0.750198,0.249789,0.989435,14.356816
0,merged,CC1,1.960607e+07,NaN,NaN,NaN,step_merged,2.128161,H-rich_Central_C_depletion,14.963983,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,merged,NaN,1.960607e+07,NaN,NaN,NaN,step_SN,0.000620,NS,1.295261,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
0,merged,END,1.960607e+07,NaN,NaN,NaN,step_end,0.000019,NS,1.295261,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,detached,ZAMS,0.000000e+00,47.967871,0.0,NaN,initial_cond,0.000000,H-rich_Core_H_burning,8.070842,...,NaN,NaN,NaN,NaN,0.750999,2.490000e-01,NaN,NaN,NaN,NaN
4,RLO1,CC1,3.831795e+07,70.999066,0.0,-5.203423,step_HMS_HMS,0.103373,H-rich_Central_C_depletion,3.017669,...,0.000000,0.0,0.00000,0.000000,0.507916,4.920825e-01,0.290726,0.709272,0.957426,19.352780
4,disrupted,NaN,3.831795e+07,NaN,NaN,NaN,step_SN,0.000428,NS,1.150693,...,0.000000,0.0,0.00000,0.000000,0.507916,4.920825e-01,0.290726,0.709272,0.957426,19.352780


In [11]:
pop.history['time']

,time
binary_index,
0,0.000000e+00
0,1.514959e+07
0,1.514959e+07
0,1.960607e+07
0,1.960607e+07
...,...
9,6.252890e+06
9,6.469990e+06
9,6.469990e+06


You can also check what columns are available in the history file:

In [12]:
pop.history.columns

Index(['state', 'event', 'time', 'orbital_period', 'eccentricity',
       'lg_mtransfer_rate', 'step_names', 'step_times', 'S1_state', 'S1_mass',
       'S1_log_R', 'S1_log_L', 'S1_lg_mdot', 'S1_he_core_mass',
       'S1_he_core_radius', 'S1_co_core_mass', 'S1_co_core_radius',
       'S1_center_h1', 'S1_center_he4', 'S1_surface_h1', 'S1_surface_he4',
       'S1_surf_avg_omega_div_omega_crit', 'S1_spin', 'S2_state', 'S2_mass',
       'S2_log_R', 'S2_log_L', 'S2_lg_mdot', 'S2_he_core_mass',
       'S2_he_core_radius', 'S2_co_core_mass', 'S2_co_core_radius',
       'S2_center_h1', 'S2_center_he4', 'S2_surface_h1', 'S2_surface_he4',
       'S2_surf_avg_omega_div_omega_crit', 'S2_spin'],
      dtype='object')

In [13]:
# using the select function
pop.history.select(where='index == 9')
# selecting all RLO1 states and only time and state columns
pop.history.select(where='state == RLO1', columns=['time', 'state'])
# selecting rows 10 to 16
pop.history.select(start=10, stop=16)


,state,event,time,orbital_period,eccentricity,lg_mtransfer_rate,step_names,step_times,S1_state,S1_mass,...,S2_he_core_mass,S2_he_core_radius,S2_co_core_mass,S2_co_core_radius,S2_center_h1,S2_center_he4,S2_surface_h1,S2_surface_he4,S2_surf_avg_omega_div_omega_crit,S2_spin
binary_index,,,,,,,,,,,,,,,,,,,,,
1,merged,NaN,4.812349e+06,NaN,NaN,NaN,step_SN,0.029677,BH,28.887105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,merged,END,4.812349e+06,NaN,NaN,NaN,step_end,0.000022,BH,28.887105,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,detached,ZAMS,0.000000e+00,2189.926519,0.0,NaN,initial_cond,0.000000,H-rich_Core_H_burning,8.813517,...,NaN,NaN,NaN,NaN,0.750999,2.490000e-01,NaN,NaN,NaN,NaN
2,detached,CC1,3.295882e+07,1793.336956,0.0,-33.837567,step_HMS_HMS,0.441680,H-rich_Central_C_depletion,8.781190,...,0.000000,0.0,0.0000,0.000000,0.745000,2.549603e-01,0.750999,0.248974,4.179712e-05,1.971680e-03
2,disrupted,NaN,3.295882e+07,NaN,NaN,NaN,step_SN,0.000431,NS,1.150693,...,0.000000,0.0,0.0000,0.000000,0.745000,2.549603e-01,0.750999,0.248974,4.179712e-05,1.971680e-03
2,disrupted,CC2,3.493982e+09,NaN,NaN,NaN,step_disrupted,2.419266,NS,1.150693,...,0.588606,NaN,0.5698,0.011607,0.000000,4.185672e-29,0.726118,0.273548,3.326915e-15,4.111893e-14


You might have notices while using the above functions that not all the binaries will have the same length in the history.
You can access these with `pop.history_lengths`. This information is also stored in the population file.

In [14]:
pop.history_lengths

,index
index,
0,6
1,6
2,6
3,6
4,6
5,6
6,6
7,6
8,6


## Population.oneline

`Population.oneline` provides a similar interface to accessing the DataFrame in the population file as `Population.history`, with similar functionality being available.

In [15]:
pop.oneline[5]
pop.oneline[[0,4]]
pop.oneline.select(where='index == 9')
pop.oneline.select(where='S1_mass_i > 20', columns=['state_i','S1_state_i', 'S1_mass_i', 'S2_state_i','S2_mass_i'])

,state_i,S1_state_i,S1_mass_i,S2_state_i,S2_mass_i
binary_index,,,,,
1,detached,H-rich_Core_H_burning,48.126908,H-rich_Core_H_burning,6.795163
8,detached,H-rich_Core_H_burning,46.627258,H-rich_Core_H_burning,18.857533
9,detached,H-rich_Core_H_burning,38.718052,H-rich_Core_H_burning,32.264858


In [16]:
pop.oneline.columns

Index(['state_i', 'event_i', 'time_i', 'orbital_period_i', 'eccentricity_i',
       'lg_mtransfer_rate_i', 'step_names_i', 'step_times_i', 'state_f',
       'event_f', 'time_f', 'orbital_period_f', 'eccentricity_f',
       'lg_mtransfer_rate_f', 'step_names_f', 'step_times_f', 'S1_state_i',
       'S1_mass_i', 'S1_log_R_i', 'S1_log_L_i', 'S1_lg_mdot_i',
       'S1_he_core_mass_i', 'S1_he_core_radius_i', 'S1_co_core_mass_i',
       'S1_co_core_radius_i', 'S1_center_h1_i', 'S1_center_he4_i',
       'S1_surface_h1_i', 'S1_surface_he4_i',
       'S1_surf_avg_omega_div_omega_crit_i', 'S1_spin_i', 'S1_state_f',
       'S1_mass_f', 'S1_log_R_f', 'S1_log_L_f', 'S1_lg_mdot_f',
       'S1_he_core_mass_f', 'S1_he_core_radius_f', 'S1_co_core_mass_f',
       'S1_co_core_radius_f', 'S1_center_h1_f', 'S1_center_he4_f',
       'S1_surface_h1_f', 'S1_surface_he4_f',
       'S1_surf_avg_omega_div_omega_crit_f', 'S1_spin_f',
       'S1_natal_kick_array_0', 'S1_natal_kick_array_1',
       'S1_natal_kick_a

## Population.formation_channels

You might be interested in figuring out what sort of formation pathways/channels a binary has followed through its evolution.

This is not a standard output of the population synthesis, but you can include it into the population file by calculating it. 
If you would like more detail on the initial mass transfer, you can set `mt_history=True`.

This will write the formation channels to the Population file, which can be accessed by `Population.formation_channels`.


In [17]:
pop.calculate_formation_channels(mt_history=True)

In [18]:
# the formation channels are loaded in with pop.formation_channels
pop.formation_channels

,channel_debug,channel
0,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
1,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
2,ZAMS_CC1_CC2_END,ZAMS_CC1_CC2_END
3,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
4,ZAMS_oRLO1_CC1_CC2_END,ZAMS_oRLO1_CC1_CC2_END
5,ZAMS_oRLO1_CC1_CC2_END,ZAMS_oRLO1_CC1_CC2_END
6,ZAMS_oRLO1_CC1_CC2_END,ZAMS_oRLO1_CC1_CC2_END
7,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
8,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
9,ZAMS_oRLO1_CC1_oRLO2_CC2_maxtime_END,ZAMS_oRLO1_CC1_oRLO2_CC2_maxtime_END


Next time you open this population file, the `formation_channels` will be available without having to be recalculated.

In [19]:
pop = Population('1e-04_Zsun_population.h5')
pop.formation_channels

,channel_debug,channel
0,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
1,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
2,ZAMS_CC1_CC2_END,ZAMS_CC1_CC2_END
3,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
4,ZAMS_oRLO1_CC1_CC2_END,ZAMS_oRLO1_CC1_CC2_END
5,ZAMS_oRLO1_CC1_CC2_END,ZAMS_oRLO1_CC1_CC2_END
6,ZAMS_oRLO1_CC1_CC2_END,ZAMS_oRLO1_CC1_CC2_END
7,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
8,ZAMS_oCE1_oMerging1_CC1_END,ZAMS_oCE1_oMerging1_CC1_END
9,ZAMS_oRLO1_CC1_oRLO2_CC2_maxtime_END,ZAMS_oRLO1_CC1_oRLO2_CC2_maxtime_END


## Selecting a sub-population

You might just want a small sub-selection of the full population, especially if you're working with large population and multi-metallicity runs.

The `Population.export_selection()` function will export just the indices of the binaries you're interested in into a new file.
The simulated and underlying mass will remain the same, since they are dependent on the population run.

If we select just 2 binaries and export them, we create a new population of just the binaries you're interested in.
In the [BBH analysis]() and [GRB analysis]() tutorials, we show how to perform a selection with multiple criteria and metallicities.



In [20]:
indices = [0,9]
pop.export_selection(indices, 'selected.h5', overwrite=False)    

In [21]:
selected = Population('selected.h5')
selected.mass_per_metallicity

,simulated_mass,underlying_mass,number_of_systems
0.0001,321.367361,1807.314212,2


<br/><br/>

Feel free to explore the small binary population you've just created!

If you want to learn more about population synthesis and how to build more complex models it is advised to continue with the remaining tutorials and consult the POSYDON documentation.

### Local MPI runs

To speed up population synthesis runs, you can run on a computing cluster, as described in [HPC Facilities](pop_syn), or you can distribute the population synthesis across multiple cores on your local machine using MPI.

To enable local MPI runs, go into the `population_params.ini`  and change `use_MPI` to `True`.

It's important to note that you cannot run have this option enabled for cluster runs!

We create a binary population simulation script to run the population:

In [ ]:
%%writefile script.py
from posydon.popsyn.synthetic_population import PopulationRunner

if __name__ == "__main__":
    synth_pop = PopulationRunner("./population_params.ini")
    synth_pop.evolve()

This script can be initiated using a local where `NR_processors` is the number of processors you would like to us.

In [ ]:
mpiexec -n ${NR_processors} python script.py

This will create a folder for each metallicity in the population and store output of the parallel runs in it.

You will have to concatenate these runs manually into a single population file per metallicity, which can be achieved using the following code:

In [ ]:
from posydon.popsyn.synthetic_population import PopulationRunner

synth_pop = PopulationRunner("./population_params.ini")
for pop in synth_pop.binary_populations:
    synth_pop.merge_parallel_runs(pop)